# NoteBot prototyping

Let's have a conversation about your (markdown) notes

## Setup

Notes:
* Use FAISS instead of Chroma because of [sqlite3 compatibility issue](https://docs.trychroma.com/troubleshooting#sqlite)

In [ ]:
from langchain.document_loaders import GitLoader
from langchain.text_splitter import MarkdownTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from notebot.constants import NOTE_REPO_URL, NOTES_PATH, DB_PATH
from dotenv import load_dotenv
import textwrap
from typing import Union

In [ ]:
load_dotenv()

## Load, split and ingest notes into vector store

Todo:
* [x] Persist database
* [ ] Improve splitting of documents

In [ ]:
def filter_notes(file_path: str) -> bool:
    return file_path.endswith(".md") and not file_path.endswith("README.md")

In [ ]:
loader = GitLoader(repo_path=str(NOTES_PATH), clone_url=NOTE_REPO_URL, file_filter=filter_notes)

In [ ]:
raw_docs = loader.load()

In [ ]:
splitter = MarkdownTextSplitter()

In [ ]:
docs = splitter.split_documents(raw_docs)

In [ ]:
raw_docs[-1].page_content

In [ ]:
docs[-1].page_content

In [ ]:
db = FAISS.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [ ]:
db.save_local(folder_path=str(DB_PATH))

## Configure Chain

Todo:
* Use open source model hosted on HuggingFace hub
* Tweak prompt
* [Include sources in response](https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db#conversationalretrievalchain-with-question-answering-with-sources)

In [ ]:
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), memory=memory)

## Chat with NoteBot

In [ ]:
class NoteBot:
    """The AI assistant chatting about your notes."""

    def __init__(self, chain) -> None:
        self.chain = chain

    def chat(
        self, question: str, print_result: bool = True, return_response: bool = False
    ) -> Union[None, dict]:
        response = self.chain({"question": question})
        if print_result:
            print(textwrap.fill(response["answer"], 88))
        if return_response:
            return response

In [ ]:
notebot = NoteBot(chain=chain)

In [ ]:
answer = notebot.chat("What is an embedding according to fastai?", return_response=True)

In [ ]:
answer.keys()

In [ ]:
notebot.chat("What can I do with the Whisper model from OpenAI when using LangChain?")